# Testing SARO

In [ ]:
import numpy as np
import saro
import xara
import matplotlib.pyplot as plt
import cupy as cp

from time import time
from scipy.linalg import sqrtm
import scipy.sparse as sparse

## Creating the KPO structure from a pupil text file

In [ ]:
thekpo = saro.KPO("test_data/pupil_model.txt")
a = thekpo.kpi.plot_pupil_and_uv(cmap="plasma", usesize=True)
plt.savefig("subaru_pupil_model.pdf")
plt.show(a)

## Defining basic parameters for the analysis

In [ ]:
wl = 1.6e-6
m2pix = xara.mas2rad( 16.7 * 128 / wl)#pscale supposed 16.7
thekpo.CWAVEL = wl

## Importing the test case data

In [ ]:
calibrator = np.load("test_data/calibrator.npy")
tarimage_single = np.load("test_data/tarimage_single.npy")
tarimage_series = np.load("test_data/tarimage_series.npy")
params_single = np.load("test_data/params_single.npy")
params_series = np.load("test_data/params_series.npy")
detpas = np.load("test_data/detpas.npy")

## Extracting the visibilities and kernel-phases

In [ ]:
cviscal = thekpo.extract_cvis_from_img(calibrator, m2pix)
kappacal = thekpo.kpi.KPM.dot(np.angle(cviscal))
cvistar_single = thekpo.extract_cvis_from_img(tarimage_single, m2pix)
kappasingle = thekpo.kpi.KPM.dot(np.angle(cvistar_single))
cvistar_series = np.array([thekpo.extract_cvis_from_img(tarimage_series[i], m2pix) for i in range(detpas.shape[0])])
kappa_series = thekpo.kpi.KPM.dot(np.angle(cvistar_series).T).T

## Importing the covariance matrix
There are several ways to build a covariance matrix for the data. None of them are perfect.

Here the covariance was evaluated from a cube of 10k realizations of independent phase screen and shot noise on a calibrator. It should be very close to ideal.

In [ ]:
Sigma = np.load("test_data/covk_10000_250_50_2_apod_False.npy")
Sigmacal = Sigma.copy()
assert Sigma.shape[0] == kappacal.shape[0]
plt.figure(figsize=(10,10))
plt.imshow(Sigma, cmap="inferno")

Note that this matrix is very much NOT diagonal, which shows that the kernel-phases, just like any Fourier plane observables, are heavily correlated. This is the reason we need to use the whitening transforms.

Here, we need a whitening matrix for the calibrated data, so the matrix has to be adjusted!

In [ ]:
W = sqrtm(np.linalg.inv(Sigma + Sigmacal))
Sigmas = np.array([Sigma for i in range(detpas.shape[0])])
Ws = sqrtm(np.linalg.inv(sparse.block_diag(Sigmas).toarray()))

## Colinearity maps: Single observations

In [ ]:
#resol = 80
#step = 10.
resol, gstep = thekpo.define_roi(verbose=True)
#We just want prettier maps than the minimum
step = 0.5*gstep
resol = 2 *resol

In [ ]:
thekpo.Mp = cp.array(W)
start = time()
colinmap, norm, rhos, thetas = thekpo.gpu_sadk_binary_match_map(resol,step,np.array([kappasingle-kappacal]),
                                           W=None, deltas=np.array([0.]), full_output=True, thetype=cp.float32)
single_cmap_time = time()-start
print("Computation of the map took: %.1f seconds" % (single_cmap_time))
plt.figure(figsize=(10,10))
plt.imshow(colinmap,cmap=saro.vdg)
plt.colorbar()
plt.show()

In [ ]:

#totalrot.append(dtheta)
totalrot = detpas[-1] - detpas[0]
thekpo.WlLp = W
pgrid = saro.dmap.build_grid(resol, step)
start = time()
detmap_single = thekpo.get_sadk_contrast_cpu(pgrid, 0.01,0.95,1.6e-6, W=None, dthetas=np.array([0.])).reshape(resol,resol)
single_detmap_time = time() - start
print("Computation of the map took: %.1f seconds" % (single_detmap_time))
logcontrast_single = np.log10(detmap_single)

In [ ]:
xx, yy = np.meshgrid(np.arange(-resol//2,resol//2), (np.arange(-resol//2,resol//2)))
rad = np.sqrt(xx**2 + yy**2)

lengths = np.sqrt(thekpo.kpi.UVC[:,0]**2 + thekpo.kpi.UVC[:,1]**2)
D = np.max(lengths)
b = np.min(lengths)
thekpo.rhomin = 0.5*thekpo.CWAVEL / D * 180/np.pi * 3600 * 1000
thekpo.rhomax = 0.5*thekpo.CWAVEL / b * 180/np.pi * 3600 * 1000
print("rho min",thekpo.rhomin)
print("rho max", thekpo.rhomax)
detmask = 1*((rad*step)<thekpo.rhomax)

a = plt.figure(figsize=(8,6))
plt.imshow(logcontrast_single*detmask, cmap="inferno")
plt.colorbar()
CSScal = plt.contour((logcontrast_single*detmask),levels=[1.5,2,2.5,3], cmap="Greens")
radi = plt.contour(rad*step,levels=[41],linestyles=["dashed"], colors=["white"])
plt.clabel(CSScal,  inline=1, fmt="%.1f")
plt.title(r"Contrast energy detector (95% / 1%)  in log ($dashed=1\lambda/D$)")
plt.tight_layout()
#plt.savefig("_ADK_detection_map_mag.png")
plt.show()

## Colinearity maps: Series of observations

In [ ]:
thekpo.Mp = cp.array(Ws)
start = time()
colinmap, norm, rhos, thetas = thekpo.gpu_sadk_binary_match_map(resol,step,kappa_series-kappacal[None,:],
                                           W=None, deltas=detpas, full_output=True)
series_cmap_time = time()-start
print("Computation of the map took: %.1f seconds" % (series_cmap_time))
plt.figure(figsize=(10,10))
plt.imshow(colinmap,cmap=saro.vdg)
plt.colorbar()
plt.show()

In [ ]:

#totalrot.append(dtheta)
totalrot = detpas[-1] - detpas[0]
thekpo.WlLp = Ws
pgrid = saro.dmap.build_grid(resol, step)
start = time()
detmap_series = thekpo.get_sadk_contrast_cpu(pgrid, 0.01,0.95,1.6e-6, W=None, dthetas=detpas).reshape(resol,resol)
series_detmap_time = time() - start
print("Computation of the map took: %.1f seconds" % (series_detmap_time))
logcontrast_series = np.log10(detmap_series)

In [ ]:
xx, yy = np.meshgrid(np.arange(-resol//2,resol//2), (np.arange(-resol//2,resol//2)))
rad = np.sqrt(xx**2 + yy**2)

lengths = np.sqrt(thekpo.kpi.UVC[:,0]**2 + thekpo.kpi.UVC[:,1]**2)
D = np.max(lengths)
b = np.min(lengths)
thekpo.rhomin = 0.5*thekpo.CWAVEL / D * 180/np.pi * 3600 * 1000
thekpo.rhomax = 0.5*thekpo.CWAVEL / b * 180/np.pi * 3600 * 1000
print("rho min",thekpo.rhomin)
print("rho max", thekpo.rhomax)
detmask = 1*((rad*step)<thekpo.rhomax)

a = plt.figure(figsize=(8,6))
plt.imshow(logcontrast_series*detmask, cmap="inferno")
plt.colorbar()
CSScal = plt.contour((logcontrast_series*detmask),levels=[1.5,2,2.5,3], cmap="Greens")
radi = plt.contour(rad*step,levels=[41],linestyles=["dashed"], colors=["white"])
plt.clabel(CSScal,  inline=1, fmt="%.1f")
plt.title(r"Contrast energy detector (95% / 1%)  in log ($dashed=1\lambda/D$)")
plt.tight_layout()
#plt.savefig("_ADK_detection_map_mag.png")
plt.show()

In [ ]:
L1p = thekpo.build_L_matrix(detpas.shape[0], crop=True, giveUf=False).toarray()
Lcov = L1p.dot(sparse.block_diag(Sigmas).toarray().dot(L1p.T))
Wl = sqrtm(np.linalg.inv(Lcov))
WlLp = Wl.dot(L1p)

In [ ]:
thekpo.Mp = cp.array(WlLp)
start = time()
colinmap, norm, rhos, thetas = thekpo.gpu_sadk_binary_match_map(resol,step,kappa_series,
                                           W=None, deltas=detpas, full_output=True)
sadk_cmap_time = time()-start
print("Computation of the map took: %.1f seconds" % (sadk_cmap_time))
plt.figure(figsize=(10,10))
plt.imshow(colinmap,cmap=saro.vdg)
plt.colorbar()
plt.show()

# Sensitivity maps

In [ ]:

#totalrot.append(dtheta)
totalrot = detpas[-1] - detpas[0]
thekpo.WlLp = WlLp
pgrid = saro.dmap.build_grid(resol, step)
start = time()
detmap_adk = thekpo.get_sadk_contrast_cpu(pgrid, 0.01,0.95,1.6e-6, W=None, dthetas=detpas).reshape(resol,resol)
sadk_detmap_time = time() - start
print("Computation of the map took: %.1f seconds" % (sadk_detmap_time))
logcontrast_adk = np.log10(detmap_adk)

In [ ]:
xx, yy = np.meshgrid(np.arange(-resol//2,resol//2), (np.arange(-resol//2,resol//2)))
rad = np.sqrt(xx**2 + yy**2)

lengths = np.sqrt(thekpo.kpi.UVC[:,0]**2 + thekpo.kpi.UVC[:,1]**2)
D = np.max(lengths)
b = np.min(lengths)
thekpo.rhomin = 0.5*thekpo.CWAVEL / D * 180/np.pi * 3600 * 1000
thekpo.rhomax = 0.5*thekpo.CWAVEL / b * 180/np.pi * 3600 * 1000
print("rho min",thekpo.rhomin)
print("rho max", thekpo.rhomax)
detmask = 1*((rad*step)<thekpo.rhomax)

a = plt.figure(figsize=(8,6))
plt.imshow(logcontrast_adk*detmask, cmap="inferno")
plt.colorbar()
CSScal = plt.contour((logcontrast_adk*detmask),levels=[1.5,2,2.5,3], cmap="Greens")
radi = plt.contour(rad*step,levels=[41],linestyles=["dashed"], colors=["white"])
plt.clabel(CSScal,  inline=1, fmt="%.1f")
plt.title(r"Contrast energy detector (95% / 1%)  in log ($dashed=1\lambda/D$)")
plt.tight_layout()
#plt.savefig("_ADK_detection_map_mag.png")
plt.show()

In [ ]:
L1p = thekpo.build_L_matrix(detpas.shape[0], crop=True, giveUf=False).toarray()
Lcov = L1p.dot(sparse.block_diag(Sigmas).toarray().dot(L1p.T))
Wl = sqrtm(np.linalg.inv(Lcov))
WlLp = Wl.dot(L1p)

In [ ]:
thekpo.Mp = WlLp
sglry, sglrmc, phat = thekpo.global_GLR(np.array([kappa_series-kappacal[None,:]]), dtheta=detpas,n=10, N=50)

In [ ]:
thekpo.Mp = W
sglry, sglrmc, phat = thekpo.global_GLR(np.array([kappasingle-kappacal]), dtheta=np.array([0.]),n=10, N=50)

```

KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKRMMMRKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKKKKKKKKKKKKKKKKKKKKM"`    ,    YKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKKKKKKKKKKKKKKKKKM"  ygKKKKKKKN  ]KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKKKKKKKKKKKKDKK" ,gKKKKKKKKKKKK  ]KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKKKKRMM**"**fF  *MRKKKKKKKKKKKM  #KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKM"     ;wgm#^,#KKKKNBKWm&BKKKKM  #KK5KKKKK*KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKM`   y#KKKKKKM,KKKKKKKKKKwKKKKKKM  #KKN KKM,#KKM5555KB555KKM9KKKMKM]555KRKKKKK

KM  ,#KKKKDKKKQ#KKKKKKKKKKKKKpYKK^ zKKKKN *aKKKKKN#KKKKJKKKMKN#JKKMBN]KKKKUKKKKK

Kp  1KKKKKKN9DKKKKKKKKKKKKKKKKN  y#KKKKKN NpYKKKKN#KKKKJKNC0KN]KNXMBN]KKKKUKKKKK

KN   `RKKKKKNBKKKKKKKKKKKKKKKM zm"KKKKKKN KKKwJ9KN2DDDK2KKKN8N#KKKwKN]DDDKwDDD]K

KKKNp    "*RKKNAKKKKKKKKKKKM,#KMDm"KKKKKK#KKKKKN#KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKWw,,      ""l"""*"Jwg#KKKKKW1KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKKKKKKKKKKKKN/*DWKKKKKKKKKKKK BKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKKKKKKKKKKKKKKKW[TKKKKKKKKKKM KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

KKKKKKKKKKKKKKKKKKKKKKKKNwp"fMMMM";#KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRKKRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
```

# GPU processing